<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px" />

# Excel Madness Lab!

_Author:_ Tim Book

## Our Mission
We work for a large supermarket chain, with stores in 10 major cities that happen to coincide with General Assembly campuses. However, this company's idea of a "database" is just a bunch of Excel spreadsheets! In order to analyze our data, we're going to need to process the existing data into a form we can use. **Our end goal is to have one csv per city.**

## Cleanup Duty!
It is a hard truth that data scientists spend a large majority of their time cleaning data. Data never arrives on our desks in exactly the format in which we want it, and it's up to us to transform it to a workable format.

Being good cleaning, moving, and reshaping data is in itself a valuable and employable job skill. If you follow these directions exactly, we will walk through constructing an automated process for processing data from this supermarket chain.

# Part I: Processing

### Step 1: Imports and the `os` library
We're going to import three libraries: numpy, pandas, and `os`.

In [1]:
# Import libraries here.
import pandas as pd
import numpy as np
import os


The `os` library is extremely useful for performing system commands from within Python. Let's get two pieces of overhead out of the way now:

1. Create an `output` folder using `os.mkdir()`
2. Create a variable called `files` that is the list of files in the `data` folder using `os.listdir()`

**WARNING:** The `os.mkdir()` function will give you an error if you try to make a folder that already exists!

In [20]:
path = '/Users/j_b/GA_DSI_works/2.02-lab-excel-madness/data'

# Create an output folder.
output_folder_test = os.mkdir('/Users/j_b/GA_DSI_works/2.02-lab-excel-madness/output_folder_test')

In [23]:
files = os.listdir(path)
files

['Jan 23.xlsx',
 'Jan 19.xlsx',
 'Jan 15.xlsx',
 'Jan 7.xlsx',
 'Jan 6.xlsx',
 'Jan 14.xlsx',
 'Jan 18.xlsx',
 'Jan 22.xlsx',
 'Jan 29.xlsx',
 'Jan 13.xlsx',
 'Jan 1.xlsx',
 'Jan 25.xlsx',
 'Jan 24.xlsx',
 'Jan 12.xlsx',
 'Jan 28.xlsx',
 'Jan 3.xlsx',
 'Jan 11.xlsx',
 'Jan 27.xlsx',
 'Jan 31.xlsx',
 'Jan 30.xlsx',
 'Jan 26.xlsx',
 'Jan 10.xlsx',
 'Jan 2.xlsx',
 'Jan 9.xlsx',
 'Jan 21.xlsx',
 'Jan 5.xlsx',
 'Jan 17.xlsx',
 'Jan 16.xlsx',
 'Jan 4.xlsx',
 'Jan 20.xlsx',
 'Jan 8.xlsx']

### Step 2: Process one data frame
It looks like we have data for the month of January. 31 files of 10 sheets each! Luckily they are all in the same format. So let's read just one in and process that. It might be helpful to open one up in your spreadsheet viewer of choice first (Excel, Numbers, Sheets, etc.)

In [51]:
# Read in data from your city from January 1st.

path = '/Users/j_b/GA_DSI_works/2.02-lab-excel-madness/data/Jan 1.xlsx'

jan_1st = pd.read_excel(path,sheet_name='Atlanta')

jan_1st.head()

,prodcode,price_eu,weight_kg,quantity
0,4272,1.168366,4.629459,155
1,4404,1.996501,2.155133,325
2,4131,2.021499,8.381480,418
3,4404,2.980396,7.450484,177
4,4650,0.977975,8.094924,384


### Step 2a: Convert to 'Merican columns
For whatever reason, our data are stored in euros and kilograms. Create `price_usd` and `weight_lb` columns. There are 2.2 pounds per kilogram, and 1.1 dollars per euro.

In [52]:
jan_1st['price_usd'] = jan_1st['price_eu'] * 1.1 
jan_1st['weight_lb'] = jan_1st['weight_kg'] * 2.2

newdf = jan_1st.sort_values(by=['prodcode'],ascending=True)
newdf['plu_code'] = newdf['prodcode']

newdf.head()

,prodcode,price_eu,weight_kg,quantity,price_usd,weight_lb,plu_code
177,3302,1.160077,3.228085,278,1.276085,7.101786,3302
74,3302,2.105314,2.991220,280,2.315845,6.580684,3302
95,3302,0.926574,4.796309,193,1.019232,10.551880,3302
158,3302,1.591874,1.338830,404,1.751061,2.945427,3302
53,3302,0.924821,9.883335,122,1.017303,21.743337,3302


### Step 2b: Merge in product names
You'll notice we also have a `plu-codes.csv` file containing actual product names matched up against their price lookup (PLU) codes. Let's merge these product names onto our Jan 1 data.
* _Hint 1:_ What kind of merge is this? Right, left, inner, outer, etc.?
* _Hint 2:_ Pay special attention to column names!

In [53]:
plu = pd.read_csv("plu-codes.csv")
plu_s = plu.sort_values(by=['plu_code'],ascending=True)
plu_s.head()

,product,plu_code
2,Apricot,3302
4,Banana,4011
16,Orange,4012
11,Kiwi,4030
8,Cucumber,4062


In [60]:
mergdf = pd.merge(newdf, plu_s, on='plu_code', how='outer')

### Step 2c: Drop unnecessary columns
We've created some extraneous columns. Drop the old price and weight columns, as well as any redundant columns.

In [79]:
mergdf = mergdf.drop(columns=['price_eu','weight_kg'])

In [80]:
mergdf = mergdf.drop(columns=['prodcode'])

In [81]:
mergdf.head()

,quantity,price_usd,weight_lb,plu_code,product,date
0,278,1.276085,7.101786,3302,Apricot,Jan 1
1,280,2.315845,6.580684,3302,Apricot,Jan 1
2,193,1.019232,10.551880,3302,Apricot,Jan 1
3,404,1.751061,2.945427,3302,Apricot,Jan 1
4,122,1.017303,21.743337,3302,Apricot,Jan 1


### Step 2d: Add the date
Simply create a new `date` column that is the date this data was collected. For example, if this is from `Jan 1.xlsx`, this column should be full of `Jan 1`.

In [82]:
mergdf['date'] = path.split(sep='.xlsx')[0][-5:]


,quantity,price_usd,weight_lb,plu_code,product,date
0,278,1.276085,7.101786,3302,Apricot,Jan 1
1,280,2.315845,6.580684,3302,Apricot,Jan 1
2,193,1.019232,10.551880,3302,Apricot,Jan 1
3,404,1.751061,2.945427,3302,Apricot,Jan 1
4,122,1.017303,21.743337,3302,Apricot,Jan 1
...,...,...,...,...,...,...
182,178,0.688542,6.951883,4958,Lemon,Jan 1
183,118,2.591707,9.547051,4958,Lemon,Jan 1
184,363,1.163053,7.466979,4958,Lemon,Jan 1
185,234,2.282065,4.387770,4958,Lemon,Jan 1


In [83]:
mergdf.head()

,quantity,price_usd,weight_lb,plu_code,product,date
0,278,1.276085,7.101786,3302,Apricot,Jan 1
1,280,2.315845,6.580684,3302,Apricot,Jan 1
2,193,1.019232,10.551880,3302,Apricot,Jan 1
3,404,1.751061,2.945427,3302,Apricot,Jan 1
4,122,1.017303,21.743337,3302,Apricot,Jan 1


In [ ]:
#df.merge(plu, left_on='prodcode', right_on='plu_code')

### Step 3: Write a function that conducts all of Step 2
This function should import a **filename and a city name** and return a fully processed DataFrame. That is, the function should:
1. Read in the data from the given file and city.
1. Create USD and pound columns.
1. Merge in product names.
1. Drop unnecessary columns.
1. Add a date column

In [95]:
def process_data(file, city):
    """
    Loads data for the specified file and city 
    """
    # load data file into a dataframe
    df = pd.read_excel(file,sheet_name=city)
    
    #merica cols FTW
    df['price_usd'] = df['price_eu'] * 1.1 
    df['weight_lb'] = df['weight_kg'] * 2.2

    #getting stuff ready for a clean merg
    sortdf = df.sort_values(by=['prodcode'],ascending=True)
    sortdf['plu_code'] = sortdf['prodcode']
    
    plu = pd.read_csv("plu-codes.csv")
    plu_s = plu.sort_values(by=['plu_code'],ascending=True)
    
    #le merge
    mergdf_first = pd.merge(sortdf, plu_s, on='plu_code', how='outer')
    mergdf = mergdf_first.drop(columns=['price_eu', 'weight_kg', 'prodcode'])
    mergdf['date'] = file.split(sep='.xlsx')[0][-5:]
    
    return mergdf

Test your function out on a new file and city!

In [102]:
file = '/Users/j_b/GA_DSI_works/2.02-lab-excel-madness/data/Jan 16.xlsx'
city = 'Atlanta'
process_data(file,city)

,quantity,price_usd,weight_lb,plu_code,product,date
0,479,1.616907,6.612895,3302,Apricot,an 16
1,306,1.950862,15.840079,3302,Apricot,an 16
2,420,2.923050,10.727157,3302,Apricot,an 16
3,480,1.915078,20.045536,3302,Apricot,an 16
4,236,1.051674,21.583158,3302,Apricot,an 16
...,...,...,...,...,...,...
186,227,1.154385,20.270743,4958,Lemon,an 16
187,309,1.130762,11.593734,4958,Lemon,an 16
188,343,1.387917,2.659086,4958,Lemon,an 16
189,481,0.996091,21.909231,4958,Lemon,an 16


In [104]:
path = '/Users/j_b/GA_DSI_works/2.02-lab-excel-madness/data'
files = os.listdir(path)
files

['Jan 23.xlsx',
 'Jan 19.xlsx',
 'Jan 15.xlsx',
 'Jan 7.xlsx',
 'Jan 6.xlsx',
 'Jan 14.xlsx',
 'Jan 18.xlsx',
 'Jan 22.xlsx',
 'Jan 29.xlsx',
 'Jan 13.xlsx',
 'Jan 1.xlsx',
 'Jan 25.xlsx',
 'Jan 24.xlsx',
 'Jan 12.xlsx',
 'Jan 28.xlsx',
 'Jan 3.xlsx',
 'Jan 11.xlsx',
 'Jan 27.xlsx',
 'Jan 31.xlsx',
 'Jan 30.xlsx',
 'Jan 26.xlsx',
 'Jan 10.xlsx',
 'Jan 2.xlsx',
 'Jan 9.xlsx',
 'Jan 21.xlsx',
 'Jan 5.xlsx',
 'Jan 17.xlsx',
 'Jan 16.xlsx',
 'Jan 4.xlsx',
 'Jan 20.xlsx',
 'Jan 8.xlsx']

### Step 4: Process all of January's data
For each spreadsheet, process the data and store the resulting DataFrame in one big list. **You only need to do this for your city!**

* _Hint 1:_ A listcomp would make this whole step one line of code!
* _Hint 2:_ You've already made that `files` variable to help you here.

In [107]:
#path_n = '/Users/j_b/GA_DSI_works/2.02-lab-excel-madness/data/{}'


database = [process_data(f'/Users/j_b/GA_DSI_works/2.02-lab-excel-madness/data/{file}', 'Atlanta') for file in files]



### Step 5: Concatenate all DataFrames from Step 4 into one large DataFrame
* _Hint:_ Is there a function in `pandas` that can do something like this for us? This is also just one line of code!

In [110]:
database

[     quantity  price_usd  weight_lb  plu_code  product   date
 0         193   0.809645   6.149797      3302  Apricot  an 23
 1         231   0.644674   7.035970      3302  Apricot  an 23
 2         404   2.189665  12.559465      3302  Apricot  an 23
 3         414   3.190854  15.811433      3302  Apricot  an 23
 4         169   1.488887   6.572202      3302  Apricot  an 23
 ..        ...        ...        ...       ...      ...    ...
 187       267   1.227310  20.167013      4958    Lemon  an 23
 188       117   2.121493  13.110241      4958    Lemon  an 23
 189       248   0.979913  10.979644      4958    Lemon  an 23
 190       376   0.583678  19.698420      4958    Lemon  an 23
 191       371   2.483511  13.467096      4958    Lemon  an 23
 
 [192 rows x 6 columns],
      quantity  price_usd  weight_lb  plu_code  product   date
 0         271   2.821189   9.384498      3302  Apricot  an 19
 1         452   2.996906  10.566099      3302  Apricot  an 19
 2         225   2.662470  2

In [111]:
result = pd.concat(database)
result

,quantity,price_usd,weight_lb,plu_code,product,date
0,193,0.809645,6.149797,3302,Apricot,an 23
1,231,0.644674,7.035970,3302,Apricot,an 23
2,404,2.189665,12.559465,3302,Apricot,an 23
3,414,3.190854,15.811433,3302,Apricot,an 23
4,169,1.488887,6.572202,3302,Apricot,an 23
...,...,...,...,...,...,...
143,486,1.893649,14.732792,4796,Tomato,Jan 8
144,407,0.866738,3.053623,4796,Tomato,Jan 8
145,287,2.066985,4.135593,4958,Lemon,Jan 8
146,440,2.723062,7.479185,4958,Lemon,Jan 8


### Step 6: Do this for all cities, write data
Here's the big one. For each city, process and the data as in steps 3-5, and then write the data to our `output` folder. Below is a dictionary of city name to desired output file name.

Before writing your DataFrame, do the following:
* Add a `city` column
* Reorder the columns into the following order:


| city | date | product | prodcode | quantity | weight_lb | price_usd |
|---|---|---|---|---|---|---|

* _Hint:_ You can reorder DataFrame columns simply by writing over your DataFrame with itself, but specifying the specific column order with `.loc`. For example:

`print(df)`

| b | c | a |
|---|---|---|
| 1 | 2 | 3 |

`df = df.loc[:, ["a", "b", "c"]]`

`print(df)`

| a | b | c |
|---|---|---|
| 3 | 1 | 2 |


In [112]:
city_dict = {
    "Atlanta": "atl.csv",
    "Austin": "atx.csv",
    "Boston": "bos.csv",
    "Chicago": "chi.csv",
    "Denver": "den.csv",
    "Los Angeles": "lax.csv",
    "New York": "nyc.csv",
    "San Francisco": "sf.csv",
    "Seattle": "sea.csv",
    "Washington, DC": "dc.csv"
}

In [ ]:
database = [process_data(f'/Users/j_b/GA_DSI_works/2.02-lab-excel-madness/data/{file}', 
                         'Atlanta') for file in files]

In [120]:
# Loop through city_dict and carry out Step 6 here.
for k,v in city_dict.items():
    database = [process_data(f'/Users/j_b/GA_DSI_works/2.02-lab-excel-madness/data/{file}', 
                         k) for file in files]
    
    df = pd.concat(database)
    df['city'] = k
    df.to_csv(f'/Users/j_b/GA_DSI_works/2.02-lab-excel-madness/output_folder/{v}')
    

# The keys of city_dict can serve as the sheet name.
for city, filename in city_dict.items():
    current_df = pd.concat([process_data(file, city) for file in files])
    current_df['city'] = city
    current_df = current_df[col_order]
    filepath = 'output/' + filename
    current_df.to_csv(filepath, index=False)

# The values of city_dict are what you should name the output .csv files.


# If done correctly, this cell could take almost a minute to run!

# Part II: Checking our answers 
In steps very similar to the ones conducted above...
1. Loop through the files we just wrote to `output`, and read them in, collecting them all in one list
1. Combine all of those DataFrames into one large DataFrame
1. For each city, find the mean `quantity`, `weight_lb`, and `price_usd`.

If you've done everything correct, your answer should look exactly like this:

![](imgs/correct-output.png)

In [123]:
output_path = '/Users/j_b/GA_DSI_works/2.02-lab-excel-madness/output_folder'
outputfiles = os.listdir(output_path)

In [128]:

filesdf = [ pd.read_csv(f'/Users/j_b/GA_DSI_works/2.02-lab-excel-madness/output_folder/{file}') for file in outputfiles ]
    
df = pd.concat(filesdf)
    
alldf = df.groupby('city')['quantity','weight_lb','price_usd'].mean()    
    

<ipython-input-128-51869e4ac421>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  alldf = df.groupby('city')['quantity','weight_lb','price_usd'].mean()


In [129]:
alldf

,quantity,weight_lb,price_usd
city,,,
Atlanta,302.485306,11.929027,1.937980
Austin,301.048298,12.092775,1.937456
Boston,298.806536,12.063057,1.900973
Chicago,301.686186,12.101555,1.930026
Denver,301.012745,12.129850,1.932088
Los Angeles,301.531276,12.167527,1.918331
New York,299.482863,12.090294,1.912662
San Francisco,298.979895,12.237399,1.925830
Seattle,300.333694,11.925486,1.903315


,Unnamed: 0,quantity,price_usd,weight_lb,plu_code,product,date,city
0,0.0,218.0,1.394968,12.902230,3302.0,Apricot,an 23,Los Angeles
1,1.0,498.0,0.934303,3.591041,3302.0,Apricot,an 23,Los Angeles
2,2.0,466.0,1.689832,11.238440,3302.0,Apricot,an 23,Los Angeles
3,3.0,166.0,0.818175,18.855599,3302.0,Apricot,an 23,Los Angeles
4,4.0,431.0,3.079136,12.701400,3302.0,Apricot,an 23,Los Angeles
...,...,...,...,...,...,...,...,...
4586,155.0,285.0,1.454879,18.231046,4958.0,Lemon,Jan 8,Boston
4587,156.0,104.0,1.377734,2.315774,4958.0,Lemon,Jan 8,Boston
4588,157.0,336.0,0.859843,4.761910,4958.0,Lemon,Jan 8,Boston
4589,158.0,158.0,1.167321,21.826945,4958.0,Lemon,Jan 8,Boston


In [ ]:
for city, filename in city_dict.items():
    current_df = pd.concat([process_data(file, city) for file in files])
    current_df['city'] = city
    current_df = current_df[col_order]
    filepath = 'output/' + filename
    current_df.to_csv(filepath, index=False)
    


# Part III (BONUS): Get this process production-ready!
_This part of the lab is optional, but very highly recommended, as the skills developed in this part are extremely common in industry._

For this step, we're going to take this whole process and put it into a production-ready Python script. **ABSOLUTELY NONE OF THIS STEP SHOULD TAKE PLACE IN A JUPYTER NOTEBOOK! PRODUCTIONALIZED ETL (_"Extract, Transform, Load"_) CODE DOES NOT TAKE PLACE IN A JUPYTER NOTEBOOK!**

The instructions are simple: As conducted in this lab, read, transform, and export the data in our Excel files into .csv files. The code should be in a `.py` file and executable from the command line. Here are some hints and tips to guide you:

### Hints, tips, and tricks:
* A good place to start is with the code you've already written. In this notebook, you can click `File > Download as > Python (.py)` to export as a `.py` file. Most of this exercise then comes down to you cleaning this file. (There will be a lot to clean).
* Remember `os.mkdir()` will throw an error if the folder you're trying to make already exists. Maybe you should check to see if it already exists. If it already exists, what should you do? (Remember that `.csv` can be overwritten with no problem.) The functions that can help you with this are all in the `os` library.
* Remember to follow all of the Python best practices we've already learned:
    - All import statements should go at the top of your script.
    - Comment your code. Comments shouldn't explain _what_ code does, but _why_ the code does this.
    - Keep your code DRY (don't repeat yourself) as opposed to WET (write everything twice). All constants should be variables that only need to be changed once. All code should be bottled into functions so you only need to fix it once.
* Make sure not to hardcode "Jan" anywhere. The point is that this code can be run throughout the lifetime of this supermarket's project, which is likely months or years. Keep your code so that if you get February data, you only need to change one tiny piece of the script (probably a file path).